In [1]:
import geopandas as gpd
import rasterio as rio
import rasterstats as rs
from rasterstats import zonal_stats
import numpy as np
import time
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

MSOA_path = r"C:\Studie\SSML\project\statistical-gis-boundaries-london\GPKG\MSOA_2011_London_gen_MHW.gpkg"
temp_path = r"C:\Studie\SSML\project\temperature\tasmonthly\subset_data_flipped.tif"

start_time = time.time()

logging.info("Starting GeoPandas read operation...")
OA_gdf = gpd.read_file(MSOA_path, engine='pyogrio', use_arrow=True)
read_gpd_time = time.time()
logging.info(f"GeoPandas read completed in {read_gpd_time - start_time:.2f} seconds.")

logging.info("Starting RasterIO open operation...")
dataset = rio.open(temp_path)
open_rio_time = time.time()
logging.info(f"RasterIO open completed in {open_rio_time - read_gpd_time:.2f} seconds.")

stats_per_band = {}
logging.info(f"Raster dataset has {dataset.count} bands.")

for band in range(1, dataset.count + 1):
    logging.info(f"Starting zonal statistics calculation for band {band} (MSAO)...")
    band_start_time = time.time()
    stats = zonal_stats(MSOA_path, temp_path, band=band, layer='MSOA_2011_London_gen_MHW')
    band_end_time = time.time()
    logging.info(f"Zonal statistics for band {band} completed in {band_end_time - band_start_time:.2f} seconds.")
    stats_per_band[band] = stats

end_time = time.time()
logging.info(f"Total execution time: {end_time - start_time:.2f} seconds.")

# Print the results (optional)
#print(stats_per_band)

2025-03-31 08:29:13,875 - INFO - Starting GeoPandas read operation...


DataSourceError: C:\Studie\SSML\project\statistical-gis-boundaries-london\GPKG\MSOA_2011_London_gen_MHW.gpkg: No such file or directory

In [ ]:
stats_per_band[168]

[{'count': 0, 'min': None, 'max': None, 'mean': None},
 {'count': 0, 'min': None, 'max': None, 'mean': None},
 {'count': 0, 'min': None, 'max': None, 'mean': None},
 {'count': 0, 'min': None, 'max': None, 'mean': None},
 {'count': 0, 'min': None, 'max': None, 'mean': None},
 {'count': 0, 'min': None, 'max': None, 'mean': None},
 {'count': 0, 'min': None, 'max': None, 'mean': None},
 {'count': 0, 'min': None, 'max': None, 'mean': None},
 {'count': 0, 'min': None, 'max': None, 'mean': None},
 {'count': 0, 'min': None, 'max': None, 'mean': None},
 {'count': 0, 'min': None, 'max': None, 'mean': None},
 {'count': 0, 'min': None, 'max': None, 'mean': None},
 {'count': 0, 'min': None, 'max': None, 'mean': None},
 {'count': 0, 'min': None, 'max': None, 'mean': None},
 {'count': 0, 'min': None, 'max': None, 'mean': None},
 {'count': 0, 'min': None, 'max': None, 'mean': None},
 {'count': 0, 'min': None, 'max': None, 'mean': None},
 {'count': 0, 'min': None, 'max': None, 'mean': None},
 {'count':

In [ ]:
dataset.read(168)

array([[7.31152142, 7.27867757, 7.3616546 , ..., 7.26575698, 7.33922373,
        7.25510532],
       [7.3155213 , 7.44502438, 7.45076331, ..., 7.28800503, 7.39225668,
        7.31833867],
       [7.34235181, 7.40173622, 7.5400287 , ..., 7.35267467, 7.44322375,
        7.36476171],
       ...,
       [8.0422364 , 8.04909002, 8.02325479, ..., 7.26051191, 7.30800648,
        7.33018954],
       [8.03680954, 8.01708868, 7.9740718 , ..., 7.3153464 , 7.38169025,
        7.40153169],
       [8.01580625, 7.95954969, 7.88825198, ..., 7.27718667, 7.41376722,
        7.32827486]])

In [ ]:
import geopandas as gpd
import rasterio as rio
from rasterstats import zonal_stats
import logging
import time
import dask
from dask.distributed import Client, progress
import pandas as pd
from shapely.geometry import box

# Configure logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

def calculate_zonal_stats(band, geometries, raster_path):
    """Calculate zonal stats for one band with better resource handling"""
    try:
        with rio.open(raster_path) as src:
            return zonal_stats(
                geometries,
                src.read(band),
                affine=src.transform,
                nodata=src.nodata,
                stats=['mean', 'min', 'max', 'median'],
                geojson_out=True
            )
    except Exception as e:
        logging.error(f"Band {band} failed: {str(e)}")
        return None

def main():
    # Load data
    OA_gdf = gpd.read_file(MSOA_path).to_crs(epsg=27700)
    geometries = OA_gdf.geometry.tolist()  # Convert to list of Shapely objects
    
    # Initialize Dask with managed memory
    client = Client(n_workers=16, threads_per_worker=1, memory_limit='8GB')
    
    try:
        # Create delayed objects
        delayed_results = [
            dask.delayed(calculate_zonal_stats)(b, geometries, temp_path)
            for b in range(1, rio.open(temp_path).count + 1)
        ]
        
        # Compute with progress tracking
        logging.info("Starting parallel computation...")
        futures = client.compute(delayed_results)
        progress(futures)  # Real-time progress bar
        
        results = client.gather(futures)
        
        # Process results
        valid_results = [pd.DataFrame(r) for r in results if r]
        if valid_results:
            stats_df = pd.concat(valid_results, axis=1)
            stats_df.columns = [f"b{idx}_{col}" 
                              for idx, col in enumerate(stats_df.columns)]
            return pd.concat([OA_gdf, stats_df], axis=1)
            
    finally:
        client.close()

if __name__ == "__main__":
    start = time.time()
    try:
        result_gdf = main()
        print(result_gdf.head())
        logging.info(f"Completed in {time.time()-start:.1f}s")
    except Exception as e:
        logging.error(f"Main process failed: {str(e)}")

2025-03-24 16:30:45,381 - INFO - State start
2025-03-24 16:30:45,399 - INFO -   Scheduler at:     tcp://127.0.0.1:64173
2025-03-24 16:30:45,401 - INFO -   dashboard at:  http://127.0.0.1:8787/status
2025-03-24 16:30:45,402 - INFO - Registering Worker plugin shuffle
2025-03-24 16:30:45,594 - INFO -         Start Nanny at: 'tcp://127.0.0.1:64180'
2025-03-24 16:30:45,597 - INFO -         Start Nanny at: 'tcp://127.0.0.1:64188'
2025-03-24 16:30:45,599 - INFO -         Start Nanny at: 'tcp://127.0.0.1:64194'
2025-03-24 16:30:45,602 - INFO -         Start Nanny at: 'tcp://127.0.0.1:64200'
2025-03-24 16:30:45,605 - INFO -         Start Nanny at: 'tcp://127.0.0.1:64184'
2025-03-24 16:30:45,609 - INFO -         Start Nanny at: 'tcp://127.0.0.1:64190'
2025-03-24 16:30:45,613 - INFO -         Start Nanny at: 'tcp://127.0.0.1:64204'
2025-03-24 16:30:45,617 - INFO -         Start Nanny at: 'tcp://127.0.0.1:64182'
2025-03-24 16:30:45,623 - INFO -         Start Nanny at: 'tcp://127.0.0.1:64192'
2025-

    MSOA11CD                  MSOA11NM    LAD11CD               LAD11NM  \
0  E02000001        City of London 001  E09000001        City of London   
1  E02000002  Barking and Dagenham 001  E09000002  Barking and Dagenham   
2  E02000003  Barking and Dagenham 002  E09000002  Barking and Dagenham   
3  E02000004  Barking and Dagenham 003  E09000002  Barking and Dagenham   
4  E02000005  Barking and Dagenham 004  E09000002  Barking and Dagenham   

     RGN11CD RGN11NM  USUALRES  HHOLDRES  COMESTRES  POPDEN  ...  \
0  E12000007  London      7375      7187        188    25.5  ...   
1  E12000007  London      6775      6724         51    31.3  ...   
2  E12000007  London     10045     10033         12    46.9  ...   
3  E12000007  London      6182      5937        245    24.8  ...   
4  E12000007  London      8562      8562          0    72.1  ...   

                                       b494_geometry  b495_type  \
0  {'type': 'MultiPolygon', 'coordinates': [(((53...    Feature   
1  {'t

In [ ]:
import geopandas as gpd
import rasterio as rio
from rasterstats import zonal_stats
import logging
import time
import dask
from dask.distributed import Client, progress
import pandas as pd
from shapely.geometry import box

# Configure logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

def calculate_zonal_stats(band, geometries, raster_path):
    """Calculate zonal stats for one band with better resource handling"""
    try:
        with rio.open(raster_path) as src:
            logging.info(f"Processing band {band}/{src.count}")
            result = zonal_stats(
                geometries,
                src.read(band),
                affine=src.transform,
                nodata=src.nodata,
                stats=['mean'],
                geojson_out=True
            )
            logging.info(f"Completed band {band}/{src.count}")
            return {"band": band, "stats": result}
    except Exception as e:
        logging.error(f"Band {band} failed: {str(e)}")
        return None

def main():
    # Path variables
    MSOA_path = r"C:\Studie\SSML\project\statistical-gis-boundaries-london\GPKG\MSOA_2011_London_gen_MHW.gpkg"
    temp_path = r"C:\Studie\SSML\project\temperature\tasmonthly\subset_data_flipped.tif"
    
    # Load data
    OA_gdf = gpd.read_file(MSOA_path).to_crs(epsg=27700)
    geometries = OA_gdf.geometry.tolist()
    
    # Initialize Dask
    client = Client(n_workers=16, threads_per_worker=1, memory_limit='8GB')
    
    try:
        # Get band count
        with rio.open(temp_path) as src:
            band_count = src.count
            logging.info(f"Processing {band_count} bands from raster")
        
        # Create delayed objects
        delayed_results = [
            dask.delayed(calculate_zonal_stats)(b, geometries, temp_path)
            for b in range(1, band_count + 1)
        ]
        
        # Compute with progress tracking
        logging.info("Starting parallel computation...")
        futures = client.compute(delayed_results)
        progress(futures)
        
        results = client.gather(futures)
        
        # Process results
        valid_bands = []
        all_band_dfs = []
        
        for result in results:
            if result:
                band_num = result["band"]
                stats_list = result["stats"]
                # Convert to DataFrame and extract mean
                stats_df = pd.DataFrame(stats_list)
                props_df = pd.json_normalize(stats_df['properties'])
                mean_series = props_df['mean'].rename(f"band_{band_num}")
                all_band_dfs.append(mean_series)
                valid_bands.append(band_num)
        
        logging.info(f"Successfully processed bands: {sorted(valid_bands)}")
        
        if all_band_dfs:
            # Combine all band columns
            all_bands_df = pd.concat(all_band_dfs, axis=1)
            # Merge with original geodataframe
            result_gdf = OA_gdf.join(all_bands_df)
            return result_gdf
        else:
            logging.warning("No valid results were returned")
            return None
    
    finally:
        client.close()

if __name__ == "__main__":
    start = time.time()
    try:
        result_gdf = main()
        if result_gdf is not None:
            # Display summary info
            logging.info(f"Result GeoDataFrame has {len(result_gdf)} rows.")
            # The default .head() will only show the first 5 rows:
            print(result_gdf.head(10))  # Show 10 rows to illustrate more output
            logging.info(f"Completed in {time.time() - start:.1f}s")
        else:
            logging.warning("result_gdf is None.")
    except Exception as e:
        logging.error(f"Main process failed: {str(e)}")


2025-03-24 17:12:03,247 - INFO - State start
2025-03-24 17:12:03,264 - INFO -   Scheduler at:     tcp://127.0.0.1:65036
2025-03-24 17:12:03,264 - INFO -   dashboard at:  http://127.0.0.1:8787/status
2025-03-24 17:12:03,268 - INFO - Registering Worker plugin shuffle
2025-03-24 17:12:03,458 - INFO -         Start Nanny at: 'tcp://127.0.0.1:65061'
2025-03-24 17:12:03,462 - INFO -         Start Nanny at: 'tcp://127.0.0.1:65053'
2025-03-24 17:12:03,467 - INFO -         Start Nanny at: 'tcp://127.0.0.1:65047'
2025-03-24 17:12:03,469 - INFO -         Start Nanny at: 'tcp://127.0.0.1:65063'
2025-03-24 17:12:03,471 - INFO -         Start Nanny at: 'tcp://127.0.0.1:65065'
2025-03-24 17:12:03,473 - INFO -         Start Nanny at: 'tcp://127.0.0.1:65051'
2025-03-24 17:12:03,478 - INFO -         Start Nanny at: 'tcp://127.0.0.1:65039'
2025-03-24 17:12:03,482 - INFO -         Start Nanny at: 'tcp://127.0.0.1:65059'
2025-03-24 17:12:03,486 - INFO -         Start Nanny at: 'tcp://127.0.0.1:65041'
2025-

    MSOA11CD                  MSOA11NM    LAD11CD               LAD11NM  \
0  E02000001        City of London 001  E09000001        City of London   
1  E02000002  Barking and Dagenham 001  E09000002  Barking and Dagenham   
2  E02000003  Barking and Dagenham 002  E09000002  Barking and Dagenham   
3  E02000004  Barking and Dagenham 003  E09000002  Barking and Dagenham   
4  E02000005  Barking and Dagenham 004  E09000002  Barking and Dagenham   
5  E02000007  Barking and Dagenham 006  E09000002  Barking and Dagenham   
6  E02000008  Barking and Dagenham 007  E09000002  Barking and Dagenham   
7  E02000009  Barking and Dagenham 008  E09000002  Barking and Dagenham   
8  E02000010  Barking and Dagenham 009  E09000002  Barking and Dagenham   
9  E02000011  Barking and Dagenham 010  E09000002  Barking and Dagenham   

     RGN11CD RGN11NM  USUALRES  HHOLDRES  COMESTRES  POPDEN  ...  band_159  \
0  E12000007  London      7375      7187        188    25.5  ...  8.520462   
1  E12000007  Lond

In [ ]:
logging.info(f"Number of rows in result_gdf: {len(result_gdf)}")
print("Shape of the resulting GeoDataFrame:", result_gdf.shape)


2025-03-24 17:20:40,565 - INFO - Number of rows in result_gdf: 983


Shape of the resulting GeoDataFrame: (983, 181)


In [ ]:
result_gdf[['band_1','band_2','band_3','band_4','band_5','band_6','band_7','band_8','band_9','band_10','band_11','band_12',]].head(1000)

,band_1,band_2,band_3,band_4,band_5,band_6,band_7,band_8,band_9,band_10,band_11,band_12
0,2.537140,4.497098,7.745181,10.724101,12.539414,17.452290,20.086538,17.507174,15.318437,12.071454,6.888881,1.478058
1,1.877811,3.751727,7.096611,9.929723,11.726454,16.574067,19.480792,17.030830,14.689174,11.469462,6.189139,0.616214
2,1.919731,3.791776,7.115173,9.948914,11.751196,16.602684,19.520171,17.069913,14.711288,11.496660,6.232307,0.663754
3,2.146392,4.030557,7.317204,10.133061,11.942482,16.831096,19.800480,17.328431,14.902804,11.696505,6.465300,0.895285
4,2.119352,4.005521,7.305410,10.157571,11.978774,16.842154,19.758457,17.296811,14.896007,11.683020,6.437525,0.877850
...,...,...,...,...,...,...,...,...,...,...,...,...
978,2.296905,4.257222,7.275381,10.437838,12.298713,17.007472,19.647015,17.447490,14.856212,11.828190,6.729684,1.239140
979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
980,2.495333,4.447818,7.532524,10.647287,12.514053,17.314483,19.957879,17.657144,15.113572,12.014669,6.904160,1.423600
981,2.484266,4.436473,7.501123,10.652953,12.534431,17.284590,19.934381,17.686146,15.082968,12.013225,6.899569,1.421816


In [ ]:
import geopandas as gpd
import rasterio as rio
from rasterstats import zonal_stats
import logging
import time
import dask
from dask.distributed import Client, progress
import pandas as pd
from shapely.geometry import box

# Configure logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

def calculate_zonal_stats(band, geometries, raster_path):
    """Calculate zonal stats for one band with better resource handling."""
    try:
        with rio.open(raster_path) as src:
            logging.info(f"Processing band {band}/{src.count}")
            result = zonal_stats(
                geometries,
                src.read(band),
                affine=src.transform,
                nodata=src.nodata,
                stats=['mean', 'count', 'nodata'],
                geojson_out=True,
                all_touched=True
            )
            logging.info(f"Completed band {band}/{src.count}")
            return {"band": band, "stats": result}
    except Exception as e:
        logging.error(f"Band {band} failed: {str(e)}")
        return None

def main():
    # Path variables
    MSOA_path = r"C:\Studie\SSML\project\statistical-gis-boundaries-london\GPKG\LSOA_2011_London_gen_MHW.gpkg"
    temp_path = r"C:\Studie\SSML\project\temperature\tasmonthly\subset_data_flipped.tif"
    
    # Load data
    OA_gdf = gpd.read_file(MSOA_path).to_crs(epsg=27700)
    geometries = OA_gdf.geometry.tolist()
    
    # Initialize Dask
    client = Client(n_workers=16, threads_per_worker=1, memory_limit='8GB')
    
    try:
        # Get band count
        with rio.open(temp_path) as src:
            band_count = src.count
            logging.info(f"Processing {band_count} bands from raster")
        
        # Create delayed objects for each band
        delayed_results = [
            dask.delayed(calculate_zonal_stats)(b, geometries, temp_path)
            for b in range(1, band_count + 1)
        ]
        
        # Compute with progress tracking
        logging.info("Starting parallel computation...")
        futures = client.compute(delayed_results)
        progress(futures)
        
        results = client.gather(futures)
        
        # Process results
        valid_bands = []
        all_band_dfs = []
        
        for result in results:
            if result:
                band_num = result["band"]
                stats_list = result["stats"]
                
                # Convert to DataFrame
                stats_df = pd.DataFrame(stats_list)
                props_df = pd.json_normalize(stats_df['properties'])
                
                # We now collect all three stats in separate columns
                # Rename the columns to differentiate each band's stats
                band_df = props_df[['mean', 'count', 'nodata']].copy()
                band_df.columns = [
                    f'band_{band_num}_mean',
                    f'band_{band_num}_count',
                    f'band_{band_num}_nodata'
                ]
                
                all_band_dfs.append(band_df)
                valid_bands.append(band_num)
        
        logging.info(f"Successfully processed bands: {sorted(valid_bands)}")
        
        if all_band_dfs:
            # Combine all columns for all bands
            all_bands_df = pd.concat(all_band_dfs, axis=1)
            # Merge with original GeoDataFrame
            result_gdf = OA_gdf.join(all_bands_df)
            return result_gdf
        else:
            logging.warning("No valid results were returned")
            return None
    
    finally:
        client.close()

if __name__ == "__main__":
    start = time.time()
    try:
        result_gdf_lsoa = main()
        if result_gdf_lsoa is not None:
            # Display summary info
            logging.info(f"Result GeoDataFrame has {len(result_gdf_lsoa)} rows.")
            print(result_gdf_lsoa.head(10))  # Display first 10 rows
            logging.info(f"Completed in {time.time() - start:.1f}s")
        else:
            logging.warning("result_gdf is None.")
    except Exception as e:
        logging.error(f"Main process failed: {str(e)}")


2025-03-24 17:36:46,720 - INFO - State start
2025-03-24 17:36:46,746 - INFO -   Scheduler at:     tcp://127.0.0.1:49178
2025-03-24 17:36:46,748 - INFO -   dashboard at:  http://127.0.0.1:8787/status
2025-03-24 17:36:46,749 - INFO - Registering Worker plugin shuffle
2025-03-24 17:36:46,931 - INFO -         Start Nanny at: 'tcp://127.0.0.1:49185'
2025-03-24 17:36:46,947 - INFO -         Start Nanny at: 'tcp://127.0.0.1:49191'
2025-03-24 17:36:46,950 - INFO -         Start Nanny at: 'tcp://127.0.0.1:49199'
2025-03-24 17:36:46,950 - INFO -         Start Nanny at: 'tcp://127.0.0.1:49183'
2025-03-24 17:36:46,960 - INFO -         Start Nanny at: 'tcp://127.0.0.1:49203'
2025-03-24 17:36:46,964 - INFO -         Start Nanny at: 'tcp://127.0.0.1:49207'
2025-03-24 17:36:46,969 - INFO -         Start Nanny at: 'tcp://127.0.0.1:49197'
2025-03-24 17:36:46,969 - INFO -         Start Nanny at: 'tcp://127.0.0.1:49181'
2025-03-24 17:36:46,969 - INFO -         Start Nanny at: 'tcp://127.0.0.1:49187'
2025-

    LSOA11CD                   LSOA11NM   MSOA11CD                  MSOA11NM  \
0  E01000001        City of London 001A  E02000001        City of London 001   
1  E01000002        City of London 001B  E02000001        City of London 001   
2  E01000003        City of London 001C  E02000001        City of London 001   
3  E01000005        City of London 001E  E02000001        City of London 001   
4  E01000006  Barking and Dagenham 016A  E02000017  Barking and Dagenham 016   
5  E01000007  Barking and Dagenham 015A  E02000016  Barking and Dagenham 015   
6  E01000008  Barking and Dagenham 015B  E02000016  Barking and Dagenham 015   
7  E01000009  Barking and Dagenham 016B  E02000017  Barking and Dagenham 016   
8  E01000010  Barking and Dagenham 015C  E02000016  Barking and Dagenham 015   
9  E01000011  Barking and Dagenham 016C  E02000017  Barking and Dagenham 016   

     LAD11CD               LAD11NM    RGN11CD RGN11NM  USUALRES  HHOLDRES  \
0  E09000001        City of London  E12000

In [ ]:
result_gdf.to_file("MSOA_temp.gpkg", driver="GPKG")

2025-03-24 17:45:19,655 - INFO - Created 4,835 records


In [ ]:
columns = [f"band_{i}_mean" for i in range(1,10)]
result_gdf_lsoa[columns].head(1000)


,band_1_mean,band_2_mean,band_3_mean,band_4_mean,band_5_mean,band_6_mean,band_7_mean,band_8_mean,band_9_mean
0,2.539914,4.493061,7.755641,10.725174,12.543472,17.445540,20.090016,17.496884,15.325144
1,2.531351,4.476498,7.744664,10.714826,12.540392,17.428950,20.087944,17.495960,15.317430
2,2.531351,4.476498,7.744664,10.714826,12.540392,17.428950,20.087944,17.495960,15.317430
3,2.580698,4.549876,7.783608,10.784760,12.612331,17.516338,20.148807,17.598118,15.359108
4,2.310783,4.214914,7.467648,10.399438,12.243239,17.076033,19.908851,17.476881,15.045530
...,...,...,...,...,...,...,...,...,...
995,1.983009,3.972486,7.018151,9.887731,11.631496,16.742199,19.325487,16.792782,14.779490
996,1.503416,3.367620,6.575059,9.537514,11.217846,16.180109,18.678712,16.501307,14.109929
997,1.481251,3.317119,6.554011,9.524875,11.206756,16.169659,18.675211,16.516306,14.086387
998,1.406014,3.244395,6.467512,9.437388,11.104814,16.054580,18.552693,16.418220,13.986092


In [ ]:
import geopandas as gpd
import rasterio

# Polygons bounding box
minx, miny, maxx, maxy = OA_gdf.total_bounds
print("Vector Extent:", (minx, miny, maxx, maxy))

# Raster bounding box
with rasterio.open(temp_path) as src:
    r_minx, r_miny, r_maxx, r_maxy = src.bounds
print("Raster Extent:", (r_minx, r_miny, r_maxx, r_maxy))


Vector Extent: (503574.1879689154, 155850.7979232141, 561956.6879517003, 200933.60890822468)
Raster Extent: (504000.0, 156000.0, 562000.0, 201000.0)


In [ ]:
with rasterio.open(temp_path) as src:
    print("No-data value:", src.nodata)


No-data value: 1e+20


In [ ]:
import geopandas as gpd
import rasterio as rio
from rasterstats import zonal_stats
import logging
import time
import dask
from dask.distributed import Client, progress
import pandas as pd
from shapely.geometry import box
import numpy as np
def flip_tif(path):
    dataset = rio.open(path)
    old_tif = dataset
    old_profile = dataset.profile
    new_profile = old_profile
    new_profile['transform'] = rio.Affine(1000.0, 0.0, 504000.0, 0.0, -1000.0, -156000.0)
    new_path = f"{path[:-4]}_flipped.tif"
    with rio.open(new_path, "w", **new_profile) as dest:
        for i in range(1, dataset.count + 1):
            new_array_tif = old_tif.read(i)
            new_array_tif = np.fliplr(np.flip(new_array_tif))
            dest.write(new_array_tif, indexes=i)

In [ ]:
flip_tif(r"/mnt/c/Users/Ruben/Documents/Project_crime/data/temperature/tasmonthly/tasmax/subset_data_tasmax.tif")

In [ ]:
import geopandas as gpd
import rasterio as rio
from rasterstats import zonal_stats
import logging
import time
import dask
from dask.distributed import Client, progress
import pandas as pd
from shapely.geometry import box

# Configure logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

def calculate_zonal_stats(band, geometries, raster_path):
    """Calculate zonal stats for one band with better resource handling."""
    try:
        with rio.open(raster_path) as src:
            logging.info(f"Processing band {band}/{src.count}")
            result = zonal_stats(
                geometries,
                src.read(band),
                affine=src.transform,
                nodata=src.nodata,
                stats=['mean', 'count', 'nodata'],
                geojson_out=True,
                all_touched=True
            )
            logging.info(f"Completed band {band}/{src.count}")
            return {"band": band, "stats": result}
    except Exception as e:
        logging.error(f"Band {band} failed: {str(e)}")
        return None

def main():
    # Path variables
    OA_gdf = r"/mnt/c/Users/Ruben/Documents/Project_crime/data/main/crime_data_refined.gpkg"
    temp_path = r"C:\Studie\SSML\project\temperature\tasmonthly\subset_data_flipped.tif"
    
    # Load data
    OA_gdf = gpd.read_file(OA_gdf, layer="OA").to_crs(epsg=27700)
    geometries = OA_gdf.geometry.tolist()
    
    # Initialize Dask
    client = Client(n_workers=16, threads_per_worker=1, memory_limit='16GB')
    
    try:
        # Get band count
        with rio.open(temp_path) as src:
            band_count = src.count
            logging.info(f"Processing {band_count} bands from raster")
        
        # Create delayed objects for each band
        delayed_results = [
            dask.delayed(calculate_zonal_stats)(b, geometries, temp_path)
            for b in range(1, band_count + 1)
        ]
        
        # Compute with progress tracking
        logging.info("Starting parallel computation...")
        futures = client.compute(delayed_results)
        progress(futures)
        
        results = client.gather(futures)
        
        # Process results
        valid_bands = []
        all_band_dfs = []
        
        for result in results:
            if result:
                band_num = result["band"]
                stats_list = result["stats"]
                
                # Convert to DataFrame
                stats_df = pd.DataFrame(stats_list)
                props_df = pd.json_normalize(stats_df['properties'])
                
                # We now collect all three stats in separate columns
                # Rename the columns to differentiate each band's stats
                band_df = props_df[['mean', 'count', 'nodata']].copy()
                band_df.columns = [
                    f'band_{band_num}_mean',
                    f'band_{band_num}_count',
                    f'band_{band_num}_nodata'
                ]
                
                all_band_dfs.append(band_df)
                valid_bands.append(band_num)
        
        logging.info(f"Successfully processed bands: {sorted(valid_bands)}")
        
        if all_band_dfs:
            # Combine all columns for all bands
            all_bands_df = pd.concat(all_band_dfs, axis=1)
            # Merge with original GeoDataFrame
            result_gdf = OA_gdf.join(all_bands_df)
            return result_gdf
        else:
            logging.warning("No valid results were returned")
            return None
    
    finally:
        client.close()

if __name__ == "__main__":
    start = time.time()
    try:
        result_gdf_oa = main()
        if result_gdf_oa is not None:
            # Display summary info
            logging.info(f"Result GeoDataFrame has {len(result_gdf_oa)} rows.")
            print(result_gdf_oa.head(10))  # Display first 10 rows
            logging.info(f"Completed in {time.time() - start:.1f}s")
            result_gdf_lsoa.to_file("OA_temp.gpkg", driver="GPKG")
        else:
            logging.warning("result_gdf is None.")
    except Exception as e:
        logging.error(f"Main process failed: {str(e)}")


2025-03-31 01:07:10,892 - INFO - Processing 168 bands from raster
2025-03-31 01:07:23,694 - INFO - Starting parallel computation...
/home/ajruben/miniforge3/envs/rapids-25.02/lib/python3.11/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 26.05 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
2025-03-31 01:10:02,675 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute('calculate_zonal_stats-99bb1f63-a8f7-418c-9b0c-7d1057dda968')" coro=<Worker.execute() done, defined at /home/ajruben/miniforge3/envs/rapids-25.02/lib/python3.11/site-packages/distributed/worker_state_machine.py:3606>> ended with CancelledError
2025-03-31 01:10:02,675 - distributed.worker.state_machine - WAR

KeyboardInterrupt: 

In [1]:
import geopandas as gpd
import rasterio as rio
from rasterstats import zonal_stats
import logging
import time
import dask
# Corrected import
from dask.distributed import Client, progress, Future, as_completed # Added as_completed
from typing import List, Dict, Optional, Any, Tuple
import pandas as pd
from shapely.geometry.base import BaseGeometry
from affine import Affine
import gc # Import garbage collector

# Configure logging (same as before)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# calculate_zonal_stats_for_band function
def calculate_zonal_stats_for_band(
    geometries: List[BaseGeometry],
    raster_path: str,
    affine: Affine,
    nodata: Optional[float],
    band: int,
    total_bands: int
) -> Optional[Dict[str, Any]]:
    """Calculates zonal statistics (MEAN ONLY) for a single band."""
    try:
        # Minimal logging inside worker
        # logging.info(f"Worker processing band {band}/{total_bands}")
        result = zonal_stats(
            geometries, raster_path, affine=affine, nodata=nodata,
            stats=['mean'], geojson_out=True, all_touched=True, band=band
        )
        # Explicitly delete large intermediate objects if needed, although result is returned directly
        # del some_large_object
        # gc.collect() # Optional: Force garbage collection if suspecting leaks within the task
        # logging.info(f"Worker completed band {band}/{total_bands}")
        return {"band": band, "stats": result}
    except Exception as e:
        # Log the error within the worker where it occurred
        logging.error(f"Worker failed on band {band}: {str(e)}", exc_info=True)
        # Force garbage collection after an error might help cleanup
        gc.collect()
        return None # Return None to indicate failure for this specific task


def main() -> Optional[gpd.GeoDataFrame]:
    """
    Main function revised to process results iteratively using as_completed,
    with corrected progress bar usage and increased memory limit.
    """
    # --- Configuration ---
    OA_GPKG_PATH = r"/mnt/c/Users/Ruben/Documents/Project_crime/data/main/crime_data_refined.gpkg"
    OA_LAYER_NAME = "OA"
    RASTER_PATH = r"/mnt/c/Users/Ruben/Documents/Project_crime/data/temperature/tasmonthly/tasmax/subset_data_tasmax_flipped.tif"
    TARGET_CRS = "EPSG:27700"
    DASK_N_WORKERS = 16
    DASK_THREADS_PER_WORKER = 1
    # == INCREASE MEMORY LIMIT ==
    # Try increasing memory, e.g., to 4GB per worker. Adjust based on your system RAM.
    # Ensure total memory (N_WORKERS * MEMORY_LIMIT) is less than your available RAM.
    DASK_MEMORY_LIMIT = '4GB'
    # ===========================
    # --- End Configuration ---

    # --- Load Vector Data ---
    try:
        logging.info(f"Loading vector data from {OA_GPKG_PATH}, layer '{OA_LAYER_NAME}'")
        oa_gdf = gpd.read_file(OA_GPKG_PATH, layer=OA_LAYER_NAME)
        logging.info(f"Loaded {len(oa_gdf)} geometries.")
        # CRS Handling
        if oa_gdf.crs is None or oa_gdf.crs.to_string().upper() != TARGET_CRS:
             logging.warning(f"Vector CRS is {oa_gdf.crs}. Reprojecting to {TARGET_CRS}.")
             oa_gdf = oa_gdf.set_crs(TARGET_CRS, allow_override=True) if oa_gdf.crs is None else oa_gdf.to_crs(TARGET_CRS)
        logging.info(f"Vector data ready (CRS: {oa_gdf.crs}).")
        # Geometry Validation
        invalid_geoms = oa_gdf[~oa_gdf.geometry.is_valid]
        if not invalid_geoms.empty:
            logging.warning(f"{len(invalid_geoms)} invalid geometries detected. Attempting fix...")
            oa_gdf['geometry'] = oa_gdf.geometry.buffer(0)
            if not all(oa_gdf.geometry.is_valid):
                logging.error("Failed to fix all invalid geometries. Aborting.")
                return None
            logging.info("Invalid geometries fixed.")
        if oa_gdf.empty:
            logging.warning("Input GeoDataFrame is empty after loading/fixing.")
            return None

        geometries = oa_gdf.geometry.tolist()
        if not geometries:
            logging.error("No valid geometries found in the GeoDataFrame.")
            return None
    except Exception as e:
        logging.error(f"Failed to load/process vector data: {e}", exc_info=True)
        return None
    # --- End Load Vector Data ---

    # --- Get Raster Metadata ---
    band_count = 0
    affine_transform = None
    nodata_value = None
    raster_crs = None
    try:
        logging.info(f"Reading metadata from raster: {RASTER_PATH}")
        with rio.open(RASTER_PATH) as src:
            band_count = src.count
            affine_transform = src.transform
            nodata_value = src.nodata
            raster_crs = src.crs
            logging.info(f"Raster Details: Bands={band_count}, CRS={raster_crs}, Nodata={nodata_value}, Transform={affine_transform}")
        if band_count == 0:
            raise ValueError("Raster file has 0 bands.")
        if oa_gdf.crs != raster_crs:
             logging.warning(f"CRS mismatch! Vector CRS: {oa_gdf.crs}, Raster CRS: {raster_crs}. ")
        if affine_transform is None or not isinstance(affine_transform, Affine):
             raise ValueError("Invalid Affine transform read from raster.")

    except Exception as e:
        logging.error(f"Failed to read raster metadata: {e}", exc_info=True)
        return None
    # --- End Get Raster Metadata ---

    # --- Initialize Dask Client ---
    client: Optional[Client] = None
    geometries_future: Optional[Future] = None
    # --- Result Storage ---
    result_gdf = oa_gdf.copy()
    processed_bands_count = 0
    # ---------------------
    try:
        # Log the updated configuration
        logging.info(f"Initializing Dask client: {DASK_N_WORKERS} workers, {DASK_THREADS_PER_WORKER} threads/worker, {DASK_MEMORY_LIMIT}/worker")
        client = Client(n_workers=DASK_N_WORKERS, threads_per_worker=DASK_THREADS_PER_WORKER,
                        memory_limit=DASK_MEMORY_LIMIT, # Use updated limit
                        dashboard_address=':0')
        logging.info(f"Dask client dashboard link: {client.dashboard_link}")

        # Scatter geometries (broadcast=True is often safer if workers might restart, despite potential memory overhead)
        logging.info(f"Scattering {len(geometries)} geometries...")
        start_scatter = time.time()
        if not geometries:
             raise ValueError("Cannot scatter empty geometries list.")
        # Keep broadcast=True for now, as losing scattered data was the failure mode.
        geometries_future = client.scatter(geometries, broadcast=True)
        logging.info(f"Geometries scattered (future created) in {time.time() - start_scatter:.2f}s.")

        logging.info(f"Submitting {band_count} zonal stats tasks via client.map...")
        start_submit = time.time()
        band_numbers = range(1, band_count + 1)
        futures: List[Future] = client.map(
            calculate_zonal_stats_for_band,
            [geometries_future] * band_count,
            [RASTER_PATH] * band_count,
            [affine_transform] * band_count,
            [nodata_value] * band_count,
            band_numbers,
            [band_count] * band_count
        )
        logging.info(f"Tasks submitted via client.map in {time.time() - start_submit:.2f}s.")

        # === Progress Handling ===
        logging.info("Registering futures for progress tracking...")
        progress(futures)

        logging.info("Processing results as they complete...")
        # Iterate directly over the as_completed iterator
        for future in as_completed(futures):
            try:
                result = future.result()

                if result:
                    band_num = result["band"]
                    stats_list = result["stats"]

                    if not stats_list or not isinstance(stats_list, list) or not all('properties' in f for f in stats_list):
                         logging.warning(f"Band {band_num} returned invalid or empty stats list. Skipping join.")
                         continue

                    props_list = [feature['properties'] for feature in stats_list]
                    props_df = pd.DataFrame(props_list)

                    if not props_df.empty and 'mean' in props_df.columns:
                        if len(props_df) == len(result_gdf):
                             mean_col_name = f'band_{band_num}_mean'
                             mean_col = props_df[['mean']].rename(columns={'mean': mean_col_name})
                             result_gdf[mean_col_name] = mean_col[mean_col_name]
                             processed_bands_count += 1
                             if processed_bands_count % 50 == 0 or processed_bands_count == band_count:
                                 logging.info(f"Processed and joined band {band_num} ({processed_bands_count}/{band_count})")
                        else:
                            logging.error(f"Band {band_num} result length mismatch! Expected {len(result_gdf)}, got {len(props_df)}. Skipping join.")
                    else:
                        logging.warning(f"Band {band_num} results missing 'mean' column or were empty. Skipping join.")
                else:
                    logging.warning(f"A task completed but returned None (likely failed internally). Future: {future}")

            except Exception as e:
                logging.error(f"Error processing a completed future: {e}", exc_info=True)
            finally:
                 # Optional: Try to explicitly release future reference, might help GC
                 del future
                 # Optional: More aggressive GC in the loop if memory pressure is extreme
                 # if processed_bands_count % 20 == 0: # e.g. every 20 futures processed
                 #    gc.collect()


        logging.info(f"Finished processing loop. Successfully joined data for {processed_bands_count}/{band_count} bands.")
        # =========================================================

        if processed_bands_count == 0:
            logging.error("No bands were successfully processed and joined. Check worker logs for errors.")
            if client:
                logging.info("Shutting down Dask client due to no processed bands.")
                client.close()
                logging.info("Dask client closed.")
            return None

        # Explicit final garbage collection before returning large dataframe
        logging.info("Performing final garbage collection.")
        gc.collect()

        return result_gdf

    except Exception as e:
        logging.error(f"An error occurred during Dask setup or main processing loop: {e}", exc_info=True)
        # Try to collect garbage even after an error
        gc.collect()
        return None
    finally:
        if client:
            logging.info("Shutting down Dask client...")
            client.close()
            logging.info("Dask client closed.")


# --- Main Execution Block ---
if __name__ == "__main__":
    script_start_time = time.time()
    logging.info("--- Starting Zonal Statistics Calculation (Mean Only, Iterative Processing, Increased Memory) ---")

    result_gdf_oa: Optional[gpd.GeoDataFrame] = None
    initial_oa_columns = 0
    try:
        try:
             initial_oa_columns = len(gpd.read_file(OA_GPKG_PATH, layer=OA_LAYER_NAME).columns)
        except Exception as read_err:
             logging.warning(f"Could not read initial columns from input file: {read_err}")
             initial_oa_columns = 0

        # Explicit GC before starting main processing
        gc.collect()
        result_gdf_oa = main()

        if result_gdf_oa is not None and not result_gdf_oa.empty:
            if len(result_gdf_oa.columns) > initial_oa_columns:
                logging.info(f"Successfully generated result GeoDataFrame: {len(result_gdf_oa)} rows, {len(result_gdf_oa.columns)} columns.")
                print("\n--- Result GeoDataFrame (Head) ---")
                print(result_gdf_oa.head(min(5, len(result_gdf_oa))))
                print("\n--- Result GeoDataFrame Info ---")
                result_gdf_oa.info(memory_usage='deep') # Show memory usage too
                print("---------------------------------\n")

                output_path = "OA_temp_stats_mean_iterative_4GB.gpkg" # Indicate memory config in name
                try:
                    logging.info(f"Saving result GeoDataFrame to {output_path}...")
                    result_gdf_oa.to_file(output_path, driver="GPKG", layer="oa_zonal_stats_mean")
                    logging.info(f"Result saved successfully to {output_path}")
                except Exception as e:
                    logging.error(f"Failed to save result: {e}", exc_info=True)
            else:
                logging.warning("Processing finished, GeoDataFrame returned, but no new statistic columns were added.")
                if result_gdf_oa is not None:
                     print("\n--- Result GeoDataFrame (Head) - No new columns ---")
                     print(result_gdf_oa.head(min(5, len(result_gdf_oa))))
                     print("---------------------------------------------\n")
        elif result_gdf_oa is not None and result_gdf_oa.empty:
             logging.warning("Processing completed, but the resulting GeoDataFrame is empty.")
        else:
            logging.error("Processing failed. No valid GeoDataFrame was produced. Check logs.")

    except Exception as e:
        logging.error(f"Critical error in main execution block: {str(e)}", exc_info=True)
    finally:
        # Explicit final GC
        logging.info("Performing final cleanup.")
        # del result_gdf_oa # Delete large object reference if no longer needed
        gc.collect()

        script_end_time = time.time()
        total_seconds = script_end_time - script_start_time
        logging.info(f"--- Zonal Statistics Calculation Finished ---")
        logging.info(f"Total execution time: {total_seconds:.2f} seconds ({time.strftime('%H:%M:%S', time.gmtime(total_seconds))})")

2025-03-31 08:29:23,875 - INFO - --- Starting Zonal Statistics Calculation (Mean Only, Iterative Processing, Increased Memory) ---
2025-03-31 08:29:23,876 - WARNING - Could not read initial columns from input file: name 'OA_GPKG_PATH' is not defined
2025-03-31 08:29:23,918 - INFO - Loading vector data from /mnt/c/Users/Ruben/Documents/Project_crime/data/main/crime_data_refined.gpkg, layer 'OA'
2025-03-31 08:29:24,446 - INFO - Loaded 25053 geometries.
2025-03-31 08:29:24,446 - INFO - Vector data ready (CRS: EPSG:27700).
2025-03-31 08:29:24,486 - WARNING - 14 invalid geometries detected. Attempting fix...
2025-03-31 08:29:24,618 - INFO - Invalid geometries fixed.
2025-03-31 08:29:24,623 - INFO - Reading metadata from raster: /mnt/c/Users/Ruben/Documents/Project_crime/data/temperature/tasmonthly/tasmax/subset_data_tasmax_flipped.tif
2025-03-31 08:29:24,631 - INFO - Raster Details: Bands=168, CRS=EPSG:27700, Nodata=1e+20, Transform=| 1000.00, 0.00, 504000.00|
| 0.00,-1000.00,-156000.00|
| 